After data cleaning, lets open the file.
I've downloaded it in two formats:
* The original format (csv) - With a dictionary with the optimal types of each column to make easier open it.

* In parquet format, that is used to manage big amounts of data.

In [1]:
import pandas as pd
import numpy as np
import pickle
import pyarrow.parquet as pq


In [3]:
with open('./data/fileformat.pickle', 'rb') as handle:
    file_format = pickle.load(handle)
file_format

{'fecha_dato': 'object',
 'ncodpers': 'int32',
 'ind_empleado': 'object',
 'pais_residencia': 'object',
 'sexo': 'object',
 'age': 'int8',
 'fecha_alta': 'object',
 'ind_nuevo': 'float32',
 'antiguedad': 'int16',
 'indrel': 'float32',
 'ult_fec_cli_1t': 'object',
 'indrel_1mes': 'object',
 'tiprel_1mes': 'object',
 'indresi': 'object',
 'indext': 'object',
 'conyuemp': 'object',
 'canal_entrada': 'object',
 'indfall': 'object',
 'tipodom': 'float32',
 'nomprov': 'object',
 'ind_actividad_cliente': 'float32',
 'renta': 'float32',
 'segmento': 'object',
 'ind_ahor_fin_ult1': 'int8',
 'ind_aval_fin_ult1': 'int8',
 'ind_cco_fin_ult1': 'int8',
 'ind_cder_fin_ult1': 'int8',
 'ind_cno_fin_ult1': 'int8',
 'ind_ctju_fin_ult1': 'int8',
 'ind_ctma_fin_ult1': 'int8',
 'ind_ctop_fin_ult1': 'int8',
 'ind_ctpp_fin_ult1': 'int8',
 'ind_deco_fin_ult1': 'int8',
 'ind_deme_fin_ult1': 'int8',
 'ind_dela_fin_ult1': 'int8',
 'ind_ecue_fin_ult1': 'int8',
 'ind_fond_fin_ult1': 'int8',
 'ind_hip_fin_ult1': 'in

In [4]:
%%time

bank = pd.read_csv('./data/clean_train.csv', dtype=file_format)
bank.head()

CPU times: user 34.4 s, sys: 1.75 s, total: 36.1 s
Wall time: 34.7 s


,Unnamed: 0,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,0,2015-01-28,1375586,N,ES,H,35,2015-01-12,0.0,6,...,0,0,0,0,0,0,0,0.0,0.0,0
1,1,2015-01-28,1050611,N,ES,V,23,2012-08-10,0.0,35,...,0,0,0,0,0,0,0,0.0,0.0,0
2,2,2015-01-28,1050612,N,ES,V,23,2012-08-10,0.0,35,...,0,0,0,0,0,0,0,0.0,0.0,0
3,3,2015-01-28,1050613,N,ES,H,22,2012-08-10,0.0,35,...,0,0,0,0,0,0,0,0.0,0.0,0
4,5,2015-01-28,1050615,N,ES,H,23,2012-08-10,0.0,35,...,0,0,0,0,0,0,0,0.0,0.0,0


In [6]:
%%time

table2 = pq.read_table('./data/clean_train.parquet')
bank2 = table2.to_pandas()

CPU times: user 23 s, sys: 6.46 s, total: 29.5 s
Wall time: 8.73 s


/home/alfonso/miniconda3/envs/kaggle/lib/python3.7/site-packages/pyarrow/pandas_compat.py:708: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
/home/alfonso/miniconda3/envs/kaggle/lib/python3.7/site-packages/pyarrow/pandas_compat.py:735: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
/home/alfonso/miniconda3/envs/kaggle/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [7]:
bank2

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1375586,N,ES,H,35,2015-01-12,0.0,6,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
1,2015-01-28,1050611,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
2,2015-01-28,1050612,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
3,2015-01-28,1050613,N,ES,H,22,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
5,2015-01-28,1050615,N,ES,H,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
6,2015-01-28,1050616,N,ES,H,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
7,2015-01-28,1050617,N,ES,H,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
9,2015-01-28,1050620,N,ES,H,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
10,2015-01-28,1050621,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
12,2015-01-28,1050623,N,ES,H,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [ ]:
df['fecha_dato'] = pd.to_datetime(df['fecha_dato'], errors='coerce', infer_datetime_format=True, format="%Y-%m-%d")
df['fecha_alta'] = pd.to_datetime(df['fecha_alta'], errors='coerce', infer_datetime_format=True, format="%Y-%m-%d")